<a href="https://colab.research.google.com/github/valentinpylypchuk/AAA-ML-Project-2025-26/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Customer Personality Analysis project focuses on understanding the characteristics and behaviors of a company’s ideal customers. The goal of this analysis is to segment customers based on their purchasing habits, preferences, and demographic attributes, in order to help the company tailor its marketing strategies and product offerings to specific groups.

The dataset provides various customer-related features such as demographics, spending patterns, and product preferences. By analyzing these attributes, we aim to identify distinct customer segments and determine which profiles are most likely to respond to specific marketing campaigns or purchase particular products.

Link: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis

In [1]:
#Libraries
import pandas as pd


In [5]:
#Importing dataset
url = "https://raw.githubusercontent.com/valentinpylypchuk/AAA-ML-Project-2025-26/refs/heads/main/marketing_campaign.csv"
df = pd.read_csv(url, sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
